In [ ]:
# combining everything we have 
import cv2;
import numpy as np;
from matplotlib import pyplot as plt;
from pywt import dwt2, idwt2;
import pywt;

def find_neighbors(index, dim):
    all_neighbors = []
    x, y = index
    width, height = dim
    
    all_neighbors.append((x, y - 1))
    all_neighbors.append((x, y + 1))
    all_neighbors.append((x - 1, y))
    all_neighbors.append((x + 1, y))
    all_neighbors.append((x - 1, y - 1))
    all_neighbors.append((x + 1, y - 1))
    all_neighbors.append((x - 1, y + 1))
    all_neighbors.append((x + 1, y + 1))

    neighbors = [index]
    #     save the location that the pixel is on for comparison
    for i in all_neighbors:
        if (i[0] >= width) or (i[1] >= height):
            continue
        elif (i[0] >= 0) and (i[1] >= 0):
            neighbors.append(i)
    return neighbors

# generates the prewitt kernels which will give us the approximate second derivative of gaussian 
def kernelGen(dim, isX):
    if(isX):
        kernelx = np.ones((1,dim), dtype = int) * -1
        zeros = np.zeros((1,dim), dtype = int)
        ones = np.ones((1,dim), dtype = int)
        for num in range(0,dim-2):
            kernelx = np.concatenate((kernelx, zeros), axis = -1)
        kernelx = np.concatenate((kernelx, ones), axis = -1)
        kernelx = np.resize(kernelx, (dim,dim))
        return kernelx
    else:
        kernely =  np.ones((dim,1), dtype = int) * -1
        zeros = np.zeros((dim,1), dtype = int)
        ones = np.ones((dim,1), dtype = int)
        for num in range(0,dim-2):
            kernely = np.concatenate((kernely, zeros), axis = -1)
        kernely = np.concatenate((kernely, ones), axis = -1)
        kernely = np.resize(kernely, (dim,dim))
        return kernely
    
def SURF():
    img = cv2.imread('red_panda.jpg')
    # convert image to grayscale 
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    fSize = [9, 15, 21, 27]
    
    ndet = [] # list of numpy arrays of dim 300*300
    for size in fSize:
        kernelx = kernelGen(size, True)
        kernely = kernelGen(size, False)

        Dx = cv2.filter2D(gray, -1, kernelx)
        Dxx = cv2.filter2D(Dx, -1, kernelx)
        Dy = cv2.filter2D(gray, -1, kernely)
        Dyy = cv2.filter2D(Dy, -1, kernely)
        Dxy = cv2.filter2D(Dx, -1, kernely)
        
        # resize in order to evaluate determinant
        new_Dyy = cv2.resize(Dyy, (300,500))
        new_Dxy = cv2.resize(Dxy, (300,500))
        
        # evaluate determinant based equation given from paper
        determinant = np.dot(Dxx,new_Dyy)-(np.square(.9)*np.dot(Dxy, new_Dxy)) 
        ndet.append(determinant)
    
    det = [] # list of determinant list 
    for i in range(0, len(ndet)):
        det.append(ndet[i].tolist())
    print('len of det', len(det), 'contains', det)
    
    kp = [] # stores key points
    loc = [] # stores location of key point
    # now for the non-maximum suppression in 3*3*3 neighborhood between scales
    for index in range(0, len(det)):
        if(index == 0 or index == len(det) - 1):
            continue
        instance = det[index] # 300 * 300 square matrix
        width = len(instance[0])
        height = len(instance[1])
        instance_dim = (width, height)
        for row in range(width):
            for col in range(height):
                neighbors_to_check = []
                current = instance[row][col] 
                upper = det[index+1]
                lower = det[index-1]
                neighbors = find_neighbors((row, col), instance_dim)
                # retrieves all valid neighbors to check, upper & lower neighbor check positions will be exactly the same
                for position in neighbors:
                    neighbors_to_check.append(upper[position[0]][position[1]])
                    neighbors_to_check.append(lower[position[0]][position[1]])
                    neighbors_to_check.append(instance[position[0]][position[1]])
                # node in same position in upper and lower level also needs to be compared
                neighbors_to_check.append(upper[row][col])
                neighbors_to_check.append(lower[row][col])
                if not (current == max(neighbors_to_check)):
                    continue
                else:
                    kp.append(current)
                    loc.append((row,col))
        print('Length of keypoint locations found', len(loc))
        print('Length of location set', len(set(loc)))
        print('Location keypoints:', set(loc))
    
SURF()